In [1]:
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# # !pip install -U PyDrive
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())


In [ ]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# gauth = GoogleAuth()
# # Try to load saved client credentials
# gauth.LoadCredentialsFile("mycreds.txt")
# if gauth.credentials is None:
#     # Authenticate if they're not there
#     gauth.LocalWebserverAuth()
# elif gauth.access_token_expired:
#     # Refresh them if expired
#     gauth.Refresh()
# else:
#     # Initialize the saved creds
#     gauth.Authorize()
# # Save the current credentials to a file
# gauth.SaveCredentialsFile("mycreds.txt")

# drive = GoogleDrive(gauth)


In [ ]:
#Import authentication libraries

# from google.colab import auth
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from oauth2client.client import GoogleCredentials

In [ ]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
# folder_id='1uXkcRXuQ5fFG0S8mLW67ewj_jsqAb_Ff'

In [ ]:
# # Auto-iterate through all files in the midi folder.
# file_list = drive.ListFile({'q': "'{0}' in parents and trashed=false".format(folder_id)}).GetList()
# for file1 in file_list:
#   print('title: {0}, id: {1}'.format(file1['title'], file1['id']))

In [ ]:
# # Download all midi files
# for file1 in file_list:
#   file_id=file1['id']
#   file_title = file1['title']
#   drive.CreateFile({'id': file_id}).GetContentFile(file_title)
  

In [2]:
from os import listdir
from os.path import isfile, join

In [3]:
echo %cd%

C:\Users\Gustavo\Documents\GitHub\GAN_project_PUCP\Midinet_VG\Model_type1


In [4]:
folder ='C:/Users/Gustavo/Documents/GitHub/GAN_project_PUCP/Midinet_VG/Model_type1'


In [93]:
songs=list(filter(lambda x: 'pickle' in x, listdir(folder)))
songs

['song0_5tracks_128measures.pickle',
 'song1_5tracks_98measures.pickle',
 'song2_5tracks_103measures.pickle',
 'song3_1tracks_98measures.pickle',
 'song4_5tracks_59measures.pickle',
 'song5_5tracks_65measures.pickle',
 'song6_5tracks_72measures.pickle',
 'song7_5tracks_157measures.pickle',
 'song_1445_5tracks_34measures.pickle',
 'song_Battle1_5tracks_54measures.pickle',
 'song_Battlethemeremix_5tracks_38measures.pickle',
 'song_bkgr_freezing_furnace_5tracks_120measures.pickle',
 'song_Bricksterbots_4tracks_33measures.pickle',
 'song_CVAoS-DanceHall_5tracks_186measures.pickle',
 'song_CVAoS-TheArena_5tracks_201measures.pickle',
 'song_DE-AW2Opening_5tracks_19measures.pickle',
 'song_Donut_Plains_Remix_5tracks_100measures.pickle',
 'song_F-Zero_GP_Legend_-_Fire_Field_5tracks_98measures.pickle',
 'song_FF5_HOM2_5tracks_96measures.pickle',
 'song_FFGBAstage1_5tracks_53measures.pickle',
 'song_FZC_Silence-KM_5tracks_129measures.pickle',
 'song_gba-pkmnfrlg-profoakencounter_5tracks_42measur

In [94]:
data_songs=np.empty((0,16,128), int)
for song in songs:
    try:  
      song_opened = open(song,"rb")
      song_loaded= pickle.load(song_opened)
      #for track 1
      for i in range(len(song_loaded[0])):
        bar=np.empty((0,128), int)[None]
        for j in song_loaded[0][i]:
          note=np.zeros(128)[None][None]
          if type(j) == type(1.5):
            note[0,0,int(j)] = 1
          bar=np.append(bar,note,axis=1)
        data_songs =np.append(data_songs,bar, axis=0)
      print(np.array(data_songs).shape,'song=',song,np.array(song_loaded).shape)
    except:
        print('error parsing')
data_songs_prev =np.roll(data_songs,1,axis=0)
data_songs=data_songs[None]
data_songs_prev=data_songs_prev[None]

(128, 16, 128) song= song0_5tracks_128measures.pickle (5, 128, 16)
(226, 16, 128) song= song1_5tracks_98measures.pickle (5, 98, 16)
(329, 16, 128) song= song2_5tracks_103measures.pickle (5, 103, 16)
(427, 16, 128) song= song3_1tracks_98measures.pickle (1, 98, 16)
(486, 16, 128) song= song4_5tracks_59measures.pickle (5, 59, 16)
(551, 16, 128) song= song5_5tracks_65measures.pickle (5, 65, 16)
(623, 16, 128) song= song6_5tracks_72measures.pickle (5, 72, 16)
(780, 16, 128) song= song7_5tracks_157measures.pickle (5, 157, 16)
(814, 16, 128) song= song_1445_5tracks_34measures.pickle (5, 34, 16)
(868, 16, 128) song= song_Battle1_5tracks_54measures.pickle (5, 54, 16)
(906, 16, 128) song= song_Battlethemeremix_5tracks_38measures.pickle (5, 38, 16)
(1026, 16, 128) song= song_bkgr_freezing_furnace_5tracks_120measures.pickle (5, 120, 16)
(1059, 16, 128) song= song_Bricksterbots_4tracks_33measures.pickle (4, 33, 16)
(1245, 16, 128) song= song_CVAoS-DanceHall_5tracks_186measures.pickle (5, 186, 16)
(

In [172]:

import random
a = [1, 2, 3, 4, 5,6,7,8]
random.randint(0,len(a)-1)

3

In [95]:
print(data_songs.shape)
print(data_songs_prev.shape)
data_songs=np.transpose(data_songs, (1,0,2,3))#Real data, input for model
data_songs_prev=np.transpose(data_songs_prev, (1,0,2,3))#Real data, 1 previous bar, input for model
print(data_songs.shape)
print(data_songs_prev.shape)

(1, 5250, 16, 128)
(1, 5250, 16, 128)
(5250, 1, 16, 128)
(5250, 1, 16, 128)


In [109]:
np.save('data_songs.npy', data_songs, allow_pickle=True, fix_imports=True)
np.save('data_songs_prev.npy', data_songs, allow_pickle=True, fix_imports=True)

In [110]:
data_test = np.load('data_songs.npy')
data_test.squeeze().reshape(len(data_test),16*128)[0:500].shape


(500, 2048)

In [111]:
testo=data_test.squeeze().reshape(len(data_test),16*128)[0:500]

In [113]:
a,b = ssc_mps_modificado(testo,10,20)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\omp.py:657: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [1]:
import os
import scipy.misc
import numpy as np
from model import MidiNet #Llamando el modelo Utilizar model2 para eliminar silencios
from utils import pp, to_json, generation_test

# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

import tensorflow as tf

flags = tf.app.flags
flags.DEFINE_integer("epoch", 10, "Epoch to train [20]")
flags.DEFINE_float("learning_rate", 0.00005, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_integer("batch_size", 120, "The size of batch [120]")
flags.DEFINE_integer("output_w", 16, "The size of the output segs to produce [16]")
flags.DEFINE_integer("output_h", 128, "The size of the output note to produce [128]")
flags.DEFINE_integer("c_dim", 1, "Number of Midi track. [1]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "samples", "Dircacectory name to save the image samples [samples]")
flags.DEFINE_string("dataset", "MidiNet_vg", "The name of dataset ")
flags.DEFINE_boolean("is_train", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("is_crop", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("generation_test", False, "True for generation_test, False for nothing [False]")
flags.DEFINE_string("gen_dir", "gen", "Directory name to save the generate samples [samples]")
FLAGS = flags.FLAGS

def main(_):
    pp.pprint(flags.FLAGS.__flags)

    if not os.path.exists(FLAGS.checkpoint_dir):
        os.makedirs(FLAGS.checkpoint_dir)
    if not os.path.exists(FLAGS.sample_dir):
        os.makedirs(FLAGS.sample_dir)
    if not os.path.exists(FLAGS.gen_dir):
        os.makedirs(FLAGS.gen_dir)

    with tf.Session() as sess:
        if FLAGS.dataset == 'MidiNet_vg':
            model = MidiNet(sess, batch_size=FLAGS.batch_size, output_w=FLAGS.output_w, output_h=FLAGS.output_h, c_dim=FLAGS.c_dim,
                    dataset_name=FLAGS.dataset, is_crop=FLAGS.is_crop, checkpoint_dir=FLAGS.checkpoint_dir, sample_dir=FLAGS.sample_dir, 
                            gen_dir=FLAGS.gen_dir)
        
        if FLAGS.is_train:
            model.train(FLAGS)
        else:
            model.load(FLAGS.checkpoint_dir)

        

if __name__ == '__main__':
    tf.app.run()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


{'batch_size': <absl.flags._flag.Flag object at 0x00000235A5458630>,
 'beta1': <absl.flags._flag.Flag object at 0x000002359FA7F2B0>,
 'c_dim': <absl.flags._flag.Flag object at 0x00000235A5458748>,
 'checkpoint_dir': <absl.flags._flag.Flag object at 0x00000235A54587F0>,
 'dataset': <absl.flags._flag.Flag object at 0x00000235A5458940>,
 'epoch': <absl.flags._flag.Flag object at 0x000002359FA6DDD8>,
 'gen_dir': <absl.flags._flag.Flag object at 0x00000235A5458B38>,
 'generation_test': <absl.flags._flag.BooleanFlag object at 0x00000235A5458B70>,
 'h': <tensorflow.python.platform.app._HelpFlag object at 0x00000235A5458C50>,
 'help': <tensorflow.python.platform.app._HelpFlag object at 0x00000235A5458C50>,
 'helpfull': <tensorflow.python.platform.app._HelpfullFlag object at 0x00000235A5458CF8>,
 'helpshort': <tensorflow.python.platform.app._HelpshortFlag object at 0x00000235A5458D68>,
 'is_crop': <absl.flags._flag.BooleanFlag object at 0x00000235A5458AC8>,
 'is_train': <absl.flags._flag.Boolea

SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!tensorboard --logdir /logs

In [ ]:
# !pip install music21

In [23]:
folder_png ='C:/Users/Gustavo/Documents/GitHub/GAN_project_PUCP/Midinet_VG/Model_type1/samples'
import os
import numpy as np
import matplotlib.pyplot as plt
import music21

In [24]:
def generate_midis(png_dir):
    for f in os.listdir(png_dir):    
        image_plt = plt.imread(os.path.join(png_dir,f))
        image_arr = np.array(image_plt)
        r_song = []
        for i in range(0,640,128):
            print(i)
            r_measure = []
            for j in range(0,16):
                measure = image_arr[j][i:i+128]
                #print(len(measure))
                r_measure.append([0 if x[0]<0.65 else x[0] for x in measure])
            r_song.append(r_measure)
        new_test_stream = music21.stream.Stream()
        sup = r_song
        new_track = music21.stream.Part()
        for j in range(0,len(sup)):
            measure = sup[j]
            for k in range(0,len(measure)):
                if sum(measure[k]) == 0 :
                    new_track.append(music21.note.Rest(type='16th'))
                else:
                    note_value = np.argmax(measure[k])
                    new_track.append(music21.note.Note(note_value, type='16th'))
        new_test_stream.append(new_track)
        new_test_stream.write('midi',f.split('.png')[0]+'.mid')
        new_test_stream.show('midi') 

In [25]:
generate_midis(folder_png)

0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


...

0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512
